#### Load data (data Ingestion) ==> Create Documents ==> Devide Doc into Chunks ==> 
#### Create vectors ==> Vector Embedding ==> Store in Vector DB ==> Query on Vector DB



In [ ]:
from dotenv import load_dotenv
load_dotenv()  # this loads .env into environment variables
import os

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Data Ingestion : From the website we need to Scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [3]:
loader = WebBaseLoader("https://docs.smith.langchain.com/administration/tutorials/manage_spend")
loader

In [4]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='\n\n\n\n\nOptimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceChangelogCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset

In [5]:
## Data splits (chunks)
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200) 
Chunk_text_documents = text_splitter.split_documents(docs)
Chunk_text_documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceChangelogCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transform

In [6]:
## Convert chunks to vectors 
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000002652D66ABC0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000002652DB21AB0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [ ]:
## Store chunks of documents in vector store DB
from langchain_community.vectorstores import FAISS
vectordb = FAISS.from_documents(Chunk_text_documents,embeddings)

In [9]:
vectordb

In [ ]:
## Query from Vector DB
query="Understand current usage"
search_result = vectordb.similarity_search(query)
search_result[0].page_content

'Understand your current usage\u200b\nThe first step of any optimization process is to understand current usage. LangSmith gives two ways to do this: Usage Graph\nand Invoices.\nUsage Graph\u200b\nThe usage graph lets us examine how much of each usage based pricing metric we have consumed lately. It does not directly show\nspend (which we will see later on our draft invoice).\nWe can navigate to the Usage Graph under Settings -> Usage and Billing -> Usage Graph.\n\nWe see in the graph above that there are two usage metrics that LangSmith charges for:\n\nLangSmith Traces (Base Charge)\nLangSmith Traces (Extended Data Retention Upgrades).'

In [12]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000026578CB3EB0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000026578DF6560>, root_client=<openai.OpenAI object at 0x0000026578DDD720>, root_async_client=<openai.AsyncOpenAI object at 0x0000026578DDDD80>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_messages([
    ("user", 
    """Answer the following question based only on the provided context: 
<context>
{context}
</context>""")
])

document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Answer the following question based only on the provided context: \n<context>\n{context}\n</context>'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000026578CB3EB0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000026578DF6560>, root_client=<openai.OpenAI object at 0x0000026578DDD720>, root_async_client=<openai.AsyncOpenAI object at 0x0000026578DDDD80>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run

In [42]:
from langchain_core.documents import Document

context_doc = Document(page_content="""
Optimization 2: limit usage
In the previous section, we managed data retention settings to optimize existing spend. In this section, we will use usage limits to prevent future overspend.

LangSmith has two usage limits: total traces and extended retention traces. These correspond to the two metrics we've been tracking on our usage graph. We can use these in tandem to have granular control over spend.

To set limits, we navigate back to Settings -> Usage and Billing -> Usage configuration. There is a table at the bottom of the page that lets you set usage limits per workspace. For each workspace, the two limits appear, along with a cost estimate.
""")

response = document_chain.invoke({
    "input" : "How to configure usage limits in LangSmith to manage cost?",
    "context" : [context_doc]
})

print(response)

To prevent future overspend, you can set usage limits on total traces and extended retention traces per workspace. This is done by navigating to Settings -> Usage and Billing -> Usage configuration, where you can adjust these limits and view a cost estimate for each workspace. This approach allows for granular control over your spending by managing how much of each resource is used.


#### Retriever 
#### Input --> Retriever (Act as an Interface) --> Vector DB

In [27]:
vectordb

In [34]:
retriever_response = vectordb.as_retriever(search_kwargs={"k": 5})
retriever_response

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002653C0B40A0>, search_kwargs={'k': 5})

In [35]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever_response, document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002653C0B40A0>, search_kwargs={'k': 5}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Answer the following question based only on the provided context: \n<context>\n{context}\n</context>'), additional_kwargs={})])
            | C

In [41]:
result = retrieval_chain.invoke({"input": "How to configure usage limits in LangSmith to manage cost?"})
result["answer"]

'What is one method for preventing future overspend with LangSmith, according to the provided context? \n\nOne method for preventing future overspend with LangSmith is to set usage limits for total traces and extended retention traces per workspace. This can be done in the Usage configuration under Settings -> Usage and Billing. Setting these limits gives granular control over spend and helps manage costs effectively across different deployment stages such as Dev, Staging, and Prod.'

In [43]:
result

{'input': 'How to configure usage limits in LangSmith to manage cost?',
 'context': [Document(id='af16dffe-cb5f-430b-9cdd-4863985bf2aa', metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='your use case with LangSmith. For example, if you run evals as part of CI/CD in dev or staging, you may\nwant to be more liberal with your usage limits to avoid test failures.\nNow that our limits are set, we can see that LangSmith shows a maximum spend estimate across all workspaces:'),
  Document(id='d41049b7-d2fe-4a3f-baa3-21937c51ba8e', metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to rea